In [ ]:
import tkinter as tk
from PIL import ImageTk, Image
from tkinter import filedialog
import numpy as np
import tensorflow as tf



def load_img():
    global img, image_data
    for img_display in frame.winfo_children():
        img_display.destroy()

    image_data = filedialog.askopenfilename(initialdir="/", title="Choose an image",
                                       filetypes=(("all files", "*.*"), ("png files", "*.png")))
    basewidth = 150 # Processing image for dysplaying
    img = Image.open(image_data)
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth, hsize), Image.LANCZOS)
    img = ImageTk.PhotoImage(img)
    file_name = image_data.split('/')
    panel = tk.Label(frame, text= str(file_name[len(file_name)-1]).upper()).pack()
    panel_image = tk.Label(frame, image=img).pack()
    
def classify():
    global model
    image = tf.keras.preprocessing.image.load_img(image_data, target_size=(224, 224))
    image_array = tf.keras.utils.img_to_array(image)
    image_array = tf.expand_dims(image_array, 0) # Create a batch

    predictions = model.predict(image_array)
    score = tf.nn.softmax(predictions[0])
    label = [["Cortinarius", "Russula", "Lactarius"][np.argmax(prediction)] for prediction in predictions]
    table = tk.Label(frame, text="Top image class predictions and confidences").pack()
    for i in range(0, len(label)):
        result = tk.Label(frame,
            text= str(label[i]).upper() + ': ' + 
                str(round(np.max(score[i])*100, 3)) + '%').pack()
        
root = tk.Tk()
root.title('Mushroom Image Classifier')
#root.iconbitmap('class.ico')
root.config(bg="#1e2f3f")
root.resizable(False, False)

tit = tk.Label(root, text="Mushroom Image Classifier", padx=25, pady=6, font=("", 12), bg="#71a9c7")
tit.pack()

canvas = tk.Canvas(root, height=500, width=500, bg="#d9d9d9")
canvas.pack()

frame = tk.Frame(root, bg="#FFFFFF")
frame.place(relwidth=0.8, relheight=0.8, relx=0.1, rely=0.1)

chose_image = tk.Button(root, text='Choose Image',
                        padx=35, pady=10,
                        fg="#FFFFFF", bg="#1e2f3f", command=load_img)
chose_image.pack(side=tk.LEFT)

class_image = tk.Button(root, text='Classify Image',
                        padx=35, pady=10,
                        fg="#FFFFFF", bg="#1e2f3f", command=classify)
class_image.pack(side=tk.RIGHT)

model = tf.keras.models.load_model("model.h5")

root.mainloop()


